<a href="https://colab.research.google.com/github/jackbowley/MMM/blob/main/MeridianTests/Meridian_additive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


# **Running test linear national models through meridian**


<a name="install"></a>

## Step 0: Install


1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).


2\. Install the latest version of Meridian, and verify that GPU is available.


In [ ]:
import sys, os
IN_COLAB = ('google.colab' in sys.modules) or ('COLAB_RELEASE_TAG' in os.environ)

In [ ]:
# Install meridian: from PyPI @ latest release (robust in Colab and local Jupyter)
import sys, subprocess
pkg = "google-meridian[colab,and-cuda]" if IN_COLAB else "google-meridian"
print(f"Installing: {pkg}")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])
except Exception as e:
    print(f"pip install failed for {pkg}: {e}")

In [ ]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

if IN_COLAB:
    from google.colab import drive


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)


### Functions etc.


In [ ]:
def import_df_from_drive(filepath):
  """Imports a CSV file from Google Drive into a pandas DataFrame.

  Args:
    filepath: The full path to the CSV file in Google Drive.

  Returns:
    A pandas DataFrame containing the data from the CSV file, or None if an error
    occurs.
  """
  try:
    drive.mount('/content/drive', force_remount=False) # Force remount in case it's already mounted
    df = pd.read_csv(filepath)
    print(f"DataFrame loaded successfully from {filepath}")
    return df
  except FileNotFoundError:
    print(f"Error: The file was not found at {filepath}")
    return None
  except Exception as e:
    print(f"An error occurred while reading the CSV file: {e}")
    return None


In [ ]:
def import_df_from_local(filename, base_dir='results'):
    """Imports a CSV file from a local 'results' folder into a pandas DataFrame.

    Args:
      filename: The CSV file name inside the results folder (e.g., 'data.csv').
      base_dir: Base directory to look in (defaults to 'results').

    Returns:
      A pandas DataFrame containing the data from the CSV file, or None if an error occurs.
    """
    try:
        if not filename:
            raise ValueError("filename must be provided, e.g., 'data.csv'")
        local_path = os.path.join(base_dir, filename)
        if not os.path.isabs(local_path):
            local_path = os.path.abspath(local_path)
        df = pd.read_csv(local_path)
        print(f"DataFrame loaded successfully from {local_path}")
        return df
    except FileNotFoundError:
        print(f"Error: The file was not found at {local_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}")
        return None


<a name="load-data"></a>

## Step 1: Load the data


### 1\.1 Load Data


In [ ]:
# Define the file path to your CSV file in Google Drive.
if IN_COLAB:
    base_dir = '/content/drive/MyDrive/work/MMM'
    filename = 'data_add.csv'
    df_data = import_df_from_drive(os.path.join(base_dir, filename))
    filename= 'var_spec_add.csv'
    df_var_spec = import_df_from_drive(os.path.join(base_dir, filename))
    df_var_spec.head()
else:
    base_dir = 'results'
    filename = 'data_add.csv'
    df_data = import_df_from_local(filename, base_dir=base_dir)
    filename = 'var_spec_add.csv'
    df_var_spec = import_df_from_local(filename, base_dir=base_dir)

In [ ]:
df = df_data.copy()
df = df.dropna(subset=['svol_xf'])
df = df.rename(columns={'Date': 'time'})
start_date = '2020-06-02'
end_date = '2024-08-27'
df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]

2\. Load Data using DataFrameInputDataBuilder


In [ ]:
builder = data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
    default_kpi_column='svol_xf',
    default_revenue_per_kpi_column='price_se_xf',
)

controls = [
    "d_jan","d_feb","d_mar","d_apr","d_may","d_sep","d_oct","d_nov","d_dec",
    "school_easter","school_ht_feb","school_ht_may","school_ht_oct","payday_25",
    "bh_ny","bh_xmas","day_valentine","ww_nat_dlta_maxtemp","ww_nat_dlta_rain",
    "ww_nat_dlta_sun","rsi_nfood_vol_sa","dist_xf",
    "price_se_xf","prom_tft","pos_fsdu","comp_lor_tot","comp_no7_tot"
]

channels = [
    "m_wow_tv","m_wow_olv","m_wow_social","m_amaze_tot","m_celeb_tv","m_celeb_outdoor","m_celeb_display"
]

builder = (
    builder
        .with_kpi(df)
        .with_revenue_per_kpi(df)
        .with_controls(df, control_cols=controls)
)

builder = builder.with_media(
    df,
    media_cols=channels,
    media_spend_cols=channels,
    media_channels=channels,
)

data = builder.build()


<a name="configure-model"></a>

## Step 2: Configure the model


In [ ]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.


In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).


<a name="model-diagnostics"></a>

## Step 3: Run model diagnostics


After the model is built, you must assess convergence, debug the model if needed, and then assess the model fit.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.


In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

2\. Assess the model's fit by comparing the expected sales against the actual sales.


In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).


<a name="generate-summary"></a>

## Step 4: Generate model results & two-page output


To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.


In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive/work/MMM/'
start_date = '2021-01-05'
end_date = '2024-08-27'
mmm_summarizer.output_model_results_summary(
    'additive_mock_model_summary_output.html', filepath, start_date, end_date
)

Here is a preview of the two-page output based on the simulated data:


In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/work/MMM/additive_mock_model_summary_output.html')

For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).


Run the following codes to save the model object:


In [ ]:
file_path = '/content/drive/MyDrive/work/MMM/saved_mmm_additive.pkl'
model.save_mmm(mmm, file_path)

Run the following codes to load the saved model:


In [ ]:
mmm = model.load_mmm(file_path)